## Chaining
We can **chain** transformations and aaction to create a computation **pipeline**
Suppose we want to compute the sum of the squares
$$ \sum_{i=1}^n x_i^2 $$
where the elements $x_i$ are stored in an RDD.

### Start the `SparkContext`

In [1]:
import os
os.environ["JAVA_HOME"] = "/opt/sdkman/candidates/java/current"

In [2]:
import numpy as np
from pyspark import SparkContext

In [3]:
sc = SparkContext(master="local[4]")

In [4]:
B=sc.parallelize(np.random.randint(0,10,size=1000))
lst = B.collect()
for i in lst: 
    print(i,end=', ')

8, 2, 4, 5, 0, 1, 2, 3, 5, 5, 8, 4, 2, 1, 7, 5, 5, 4, 4, 0, 3, 4, 4, 2, 4, 2, 8, 3, 0, 8, 8, 1, 2, 0, 6, 7, 9, 2, 4, 7, 9, 6, 2, 9, 1, 2, 3, 1, 7, 1, 4, 8, 1, 9, 0, 1, 5, 2, 3, 2, 7, 5, 4, 1, 8, 5, 3, 5, 8, 7, 8, 7, 1, 7, 1, 2, 0, 8, 8, 6, 2, 3, 1, 0, 6, 9, 5, 2, 9, 1, 9, 3, 0, 0, 3, 1, 6, 1, 2, 0, 8, 1, 0, 3, 6, 8, 4, 7, 5, 4, 8, 3, 1, 7, 3, 3, 2, 5, 7, 5, 7, 5, 5, 4, 7, 3, 3, 8, 2, 1, 2, 4, 6, 7, 2, 3, 6, 7, 4, 0, 6, 2, 9, 5, 7, 4, 2, 1, 7, 3, 5, 9, 5, 6, 6, 9, 9, 5, 8, 9, 5, 4, 7, 6, 5, 6, 8, 2, 0, 0, 5, 5, 2, 4, 8, 6, 9, 3, 4, 8, 3, 1, 9, 5, 3, 7, 5, 5, 1, 7, 1, 8, 7, 5, 4, 2, 4, 7, 4, 1, 6, 3, 8, 9, 6, 9, 5, 4, 1, 6, 5, 4, 9, 8, 9, 7, 6, 9, 8, 9, 2, 8, 9, 0, 8, 1, 0, 1, 6, 5, 3, 1, 8, 6, 4, 9, 5, 7, 1, 4, 0, 0, 8, 8, 8, 5, 4, 3, 0, 8, 7, 4, 6, 6, 7, 9, 7, 3, 7, 4, 4, 1, 5, 5, 2, 8, 5, 2, 7, 1, 0, 6, 2, 9, 5, 4, 4, 7, 3, 5, 2, 4, 1, 2, 9, 7, 3, 6, 9, 2, 8, 4, 7, 6, 3, 1, 2, 4, 5, 8, 1, 6, 7, 8, 0, 5, 2, 9, 5, 6, 9, 7, 5, 0, 7, 2, 0, 9, 7, 4, 7, 5, 2, 0, 5, 6, 2, 9, 4, 1, 0, 8, 0, 4

### Sequential syntax for chaining
Perform assignment after each computation

In [5]:
%%time
Squares=B.map(lambda x:x*x)
summation = Squares.reduce(lambda x,y:x+y)

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 622 ms


In [6]:
print(summation)

29204


### Cascaded syntax for chaining
Combine computations into a single cascaded command

In [7]:
%%time
B.map(lambda x:x*x).reduce(lambda x,y:x+y)

CPU times: user 5.2 ms, sys: 2.74 ms, total: 7.93 ms
Wall time: 61.4 ms


29204

### Both syntaxes mean exactly the same thing
The only difference:
* In the sequential syntax the intermediate RDD has a name `Squares`
* In the cascaded syntax the intermediate RDD is *anonymous*

The execution is identical!

### Sequential execution
The standard way that the map and reduce are executed is
* perform the map
* store the resulting RDD in memory
* perform the reduce

### Disadvantages of Sequential execution

1. Intermediate result (`Squares`) requires memory space.
2. Two scans of memory (of `B`, then of `Squares`) - double the cache-misses.

### Pipelined execution
Perform the whole computation in a single pass. For each element of **`B`**
1. Compute the square
2. Enter the square as input to the `reduce` operation.

### Advantages of Pipelined execution

1. Less memory required - intermediate result is not stored.
2. Faster - only one pass through the Input RDD.

In [8]:
sc.stop()